# **Cosas a tener en cuenta******

Fecha de entrega: en 2 semanas

1. Nºactuals y forecast
2. Horizonte de previsión (siempre el mismo)
3. Nºpaises y productos
4. Historico de datos actuals / forecast
5. Forecast distintos
6. Rangos de Amount

# **Visualización de datos**

**Asignatura:** Visualización de datos

**Autora:** Sandra Millán Palacios



Para este proyecto de análisis y visualización de datos de ventas, realizaremos un análisis exhaustivo que nos permita explorar y entender el comportamiento de las ventas de un conjunto de datos que comprende transacciones realizadas en distintos países, a través de varias marcas y en diferentes periodos de tiempo.Para ello nos centraremos en explorar cómo se distribuyen y evolucionan las ventas de acuerdo a varias dimensiones clave:

1. Cómo se distribuyen las ventas realizadas en:

  - Cada país
  - Cada mes y año
  - Cada marca

2. Cuál es la tendencia y estacionalidad de:

  - Todas las ventas del país con menos ventas
  - La marca con más ventas

3. Cuáles son las predicciones hechas en España y cómo de
buenas son

## **Carga la base de datos**

In [1]:
pip install plotly_express

In [2]:
# Librerías necesarias

import plotly_express as px
import pandas as pd
from google.colab import files

La base de datos proporcionada consta de 18,666 registros y 8 columnas, cada una con un propósito específico relacionado con el seguimiento de ventas en distintos países, marcas, y en un periodo de tiempo detallado:

- **COUNTRY** - Indica el país donde se realizó la venta.
- **SUBBRAND** - Especifica la sub-marca vendida.
- **YEAR** - Año en que se realizó la venta o en que se realizó una predicción.
- **MONTH** - Mes en el que ocurrió la venta o la predicción.
- **SCENARIO** - Describe si el registro corresponde a una predicción o a una observación real.
- **FORECAST** - Mes en el que se realizó la predicción.
- **FORECAST_YEAR** - Año en el que se realizó la predicción.
- **AMOUNT** - Representa el volumen de ventas en una transacción. Es la métrica principal para evaluar el rendimiento de ventas, tendencias y precisión de pronósticos.

In [3]:
# Cargar la base de datos: datos_ejercicio_ventas.csv

uploaded = files.upload()

Saving datos_ejercicio_ventas.csv to datos_ejercicio_ventas.csv


In [4]:
df = pd.read_csv('/content/datos_ejercicio_ventas.csv')
df.head()

,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
0,Portugal,Lipton (L3),2023,12,AI_forecast,AI_P02F,2023.0,754356.237194
1,Great Britain,Lipton (L3),2023,12,AI_forecast,AI_P10F,2023.0,560030.558029
2,Spain,Pepsi Max (L3),2023,12,AI_forecast,AI_P09F,2023.0,88501.980847
3,Great Britain,7up (L3),2024,12,AI_forecast,AI_P10F,2023.0,363224.511516
4,Hungary,Lipton (L3),2023,9,AI_forecast,AI_P03F,2023.0,396176.120491


## **Transformación de los datos**

Para comenzar el análisis de esta base de datos de ventas, primero dividiremos los registros en dos subconjuntos: uno que contenga únicamente los datos de ventas reales y otro que incluya las predicciones. Esta división facilitará un estudio detallado y específico de cada grupo de datos.

En primer lugar, ajustaremos el formato de los valores en la columna de predicciones, transformando el formato AI_P(mes)F en un valor numérico de mes. Esto permitirá una mejor interpretación temporal de las predicciones y su comparación con los datos reales. Posteriormente, realizaremos un análisis inicial mediante el cálculo de estadísticos descriptivos para los campos numéricos (año, mes y volumen de ventas) en ambos conjuntos. Esta revisión es fundamental para evaluar la coherencia y cobertura temporal de los datos, y nos permitirá verificar si los periodos de predicción se alinean con los años de datos reales disponibles. Finalmente, determinaremos el número de países y sub-marcas representados en los datos, lo cual aportará claridad sobre la complejidad y dimensión del problema de análisis y modelado.

### **Crear dos subconjuntos de datos**

Crear dos dataset: uno de actuals (real) y el otro de Al_forecast (predicción)

In [5]:
df_actual = df[df['SCENARIO'] == 'actual']
df_forecast = df[df['SCENARIO'] == 'AI_forecast']

### **Limpiar los datos**

Primero, hemos convertido las columnas numéricas (FORECAST_YEAR) al formato entero. Luego, ajustamos la columna FORECAST en el subconjunto de predicciones, transformando su formato para que los meses donde se predijo la información estén representados numéricamente. Finalmente, calculamos los estadísticos básicos para las columnas numéricas en los datos reales y en las predicciones para observar la distribución de cada subconjunto.

In [6]:
# Primero hay que cambair a dos dataset
df_forecast['FORECAST_YEAR'] = df_forecast['FORECAST_YEAR'].astype(int)

<ipython-input-6-ca7a3dae5826>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forecast['FORECAST_YEAR'] = df_forecast['FORECAST_YEAR'].astype(int)


In [7]:
import re

def extract_number(text):
    match = re.search(r'\d+', text) # Mira lo que sea texto
    if match:
        return int(match.group(0)) - 1 #Coge lo que no sea letra, resta 1 (formato datos) a int
    else:
        return 12  # Si no hya nada es proque es el mes 12

df_forecast['FORECAST'] = df_forecast['FORECAST'].apply(extract_number)

<ipython-input-7-6c1b4e25b6f6>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_forecast['FORECAST'] = df_forecast['FORECAST'].apply(extract_number)


In [8]:
df_forecast[['YEAR', 'MONTH', 'FORECAST', 'FORECAST_YEAR', 'AMOUNT']].describe()

,YEAR,MONTH,FORECAST,FORECAST_YEAR,AMOUNT
count,17766.000000,17766.000000,17766.000000,17766.0,1.776600e+04
mean,2023.732298,6.514860,6.665653,2023.0,9.837240e+05
std,0.590992,3.472165,3.370757,0.0,1.915109e+06
min,2023.000000,1.000000,1.000000,2023.0,0.000000e+00
25%,2023.000000,3.000000,4.000000,2023.0,8.917303e+04
50%,2024.000000,7.000000,7.000000,2023.0,3.239128e+05
75%,2024.000000,10.000000,10.000000,2023.0,1.100389e+06
max,2025.000000,12.000000,12.000000,2023.0,1.373810e+07


1. **Año de Predicción:**

Los años abarcan desde 2023 hasta 2025, con una media cercana a 2024. Esto indica que el horizonte de predicción cubre aproximadamente tres años, con una mayoría concentrada en 2023 y 2024.

2. **Mes de Predicción:**

Los valores en MONTH va de 1 a 12, cubriendo todos los meses. Esto garantiza que hay datos de predicción para todos los meses del año, permitiendo una evaluación de estacionalidad.

3. **Volumen de Ventas (AMOUNT):**

La media de ventas predichas es 983,724, pero existe una alta variabilidad (desviación estándar de 1,915,109) con valores que oscilan desde 0 hasta 13,738,100. Esto indica una amplia distribución en los volúmenes de ventas esperados, posiblemente debido a variaciones entre países, productos, o estacionalidad en el tiempo. El 0 implica que no se predijo ninguna venta (observar más adelante).

4. **Consistencia Temporal:**

El valor constante en FORECAST_YEAR (2023) sugiere que todas las predicciones se han realizado con base en los datos observados de ese año, lo cual puede ser relevante al evaluar la precisión de estos pronósticos.

**Horizonte de predicción**

El horizonte de predicción es el periodo de tiempo hacia el futuro para el cual un modelo o método de pronóstico realiza sus estimaciones. En otras palabras, es la extensión temporal que cubre la predicción, definida en unidades de tiempo como días, meses, trimestres o años. En nuestro caso se basa en meses.

Para ello, hay que tener en cuanta que hablamos de una sub-marca, en un lugar, y con la predicción qu ese haya hecho en un mes en particular.

El horizonte de predicción conincide para todas las marcas, paises y mes en el que se hizo la predicción.

In [12]:
# Sacar el horizonte de predicción

def horizonte(df):
  i = 0
  for year in range(2023, 2026, 1):
    df_year = df[df['YEAR'] == year]
    for month in range(1, 13, 1):
      if month in df_year['MONTH'].values:
        i = i + 1
  return i

In [17]:
print(f'El horizonte de predicción es de {horizonte(df_forecast[(df_forecast["SUBBRAND"] == "Lipton (L3)") & (df_forecast["COUNTRY"] == "Spain") & (df_forecast["FORECAST"] == 1)])} meses : Lipton')


El horizonte de predicción es de 18 meses : Lipton


In [16]:
print(f'El horizonte de predicción es de {horizonte(df_forecast[(df_forecast["SUBBRAND"] == "7up (L3)") & (df_forecast["COUNTRY"] == "Spain") & (df_forecast["FORECAST"] == 1) & (df_forecast["FORECAST"] == 1)])} meses: 7up')

El horizonte de predicción es de 18 meses: 7up


**¿Existen varias predicciones de una misma marca en un mismo pais y para un solo mes? PREGUNTAR *********

In [20]:
df_forecast[(df_forecast["SUBBRAND"] == "Lipton (L3)") & (df_forecast["COUNTRY"] == "Spain") & (df_forecast["MONTH"] == 1) & (df_forecast["YEAR"] == 2024)]


,COUNTRY,SUBBRAND,YEAR,MONTH,SCENARIO,FORECAST,FORECAST_YEAR,AMOUNT
322,Spain,Lipton (L3),2024,1,AI_forecast,5,2023,12670.573620
2239,Spain,Lipton (L3),2024,1,AI_forecast,3,2023,12171.089747
3426,Spain,Lipton (L3),2024,1,AI_forecast,7,2023,12124.669695
4568,Spain,Lipton (L3),2024,1,AI_forecast,6,2023,12820.182303
5151,Spain,Lipton (L3),2024,1,AI_forecast,2,2023,12574.943641
8882,Spain,Lipton (L3),2024,1,AI_forecast,9,2023,13480.829252
12665,Spain,Lipton (L3),2024,1,AI_forecast,10,2023,9271.108395
14271,Spain,Lipton (L3),2024,1,AI_forecast,11,2023,9481.729039
16851,Spain,Lipton (L3),2024,1,AI_forecast,12,2023,6767.738848
16987,Spain,Lipton (L3),2024,1,AI_forecast,8,2023,11704.779148


### **Análisis de datos reales**

Vemos si los datos son coherentes:

In [10]:
pd.DataFrame(df['COUNTRY'].unique(), columns=['Unique Countries'])

,Unique Countries
0,Portugal
1,Great Britain
2,Spain
3,Hungary
4,Norway
5,Denmark
6,Netherlands
7,Italy
8,Czech


In [11]:
pd.DataFrame(df['SUBBRAND'].unique(), columns=['Unique Countries'])

,Unique Countries
0,Lipton (L3)
1,Pepsi Max (L3)
2,7up (L3)
3,Pepsi Regular (L3)
4,Mountain Dew (L3)
5,7up Free (L3)
